# Logistic regression - code along

In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("log_reg").getOrCreate()

22/12/03 15:39:10 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface wlp3s0)
22/12/03 15:39:10 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/03 15:39:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/03 15:39:14 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
df = spark.read.format("libsvm").load("../../data/sample_libsvm_data.txt")

22/12/03 15:40:45 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
df.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [7]:
logistic_regression = (
    LogisticRegression()
    .fit(df)
)

22/12/03 15:41:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/12/03 15:41:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [8]:
logistic_regression_summary = logistic_regression.summary

In [10]:
logistic_regression_summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [11]:
logistic_regression_summary.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [12]:
train, test = df.randomSplit([0.7, 0.3])

In [13]:
logistic_regression = (
    LogisticRegression()
    .fit(train)
)

In [14]:
test_predictions = logistic_regression.evaluate(test)

In [16]:
test_predictions.predictions.show(5)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[16.9525170837101...|[0.99999995658744...|       0.0|
|  0.0|(692,[122,123,148...|[22.8037101674687...|[0.99999999987512...|       0.0|
|  0.0|(692,[124,125,126...|[46.2567763143417...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[24.7350914813038...|[0.99999999998189...|       0.0|
|  0.0|(692,[126,127,128...|[40.3379463369537...|           [1.0,0.0]|       0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



In [18]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MultilabelClassificationEvaluator


In [19]:
test_roc = BinaryClassificationEvaluator().evaluate(test_predictions.predictions)